In [29]:
import yfinance as yf
from datetime import datetime

import plotly
import plotly.express as px
import pandas as pd

import dateutil.relativedelta
from datetime import datetime

import plotly.graph_objects as go

In [5]:
def psar(barsdata, iaf = 0.02, maxaf = 0.2):
    length = len(barsdata)
    dates = list(barsdata['Date'])
    high = list(barsdata['High'])
    low = list(barsdata['Low'])
    close = list(barsdata['Close'])
    psar = close[0:len(close)]
    psarbull = [None] * length
    psarbear = [None] * length
    bull = True
    af = iaf
    ep = low[0]
    hp = high[0]
    lp = low[0]
    for i in range(2,length):
        if bull:
            psar[i] = psar[i - 1] + af * (hp - psar[i - 1])
        else:
            psar[i] = psar[i - 1] + af * (lp - psar[i - 1])
        reverse = False
        if bull:
            if low[i] < psar[i]:
                bull = False
                reverse = True
                psar[i] = hp
                lp = low[i]
                af = iaf
        else:
            if high[i] > psar[i]:
                bull = True
                reverse = True
                psar[i] = lp
                hp = high[i]
                af = iaf
        if not reverse:
            if bull:
                if high[i] > hp:
                    hp = high[i]
                    af = min(af + iaf, maxaf)
                if low[i - 1] < psar[i]:
                    psar[i] = low[i - 1]
                if low[i - 2] < psar[i]:
                    psar[i] = low[i - 2]
            else:
                if low[i] < lp:
                    lp = low[i]
                    af = min(af + iaf, maxaf)
                if high[i - 1] > psar[i]:
                    psar[i] = high[i - 1]
                if high[i - 2] > psar[i]:
                    psar[i] = high[i - 2]
        if bull:
            psarbull[i] = psar[i]
        else:
            psarbear[i] = psar[i]
    return {"dates":dates, "high":high, "low":low, "close":close, "psar":psar, "psarbear":psarbear, "psarbull":psarbull}

In [14]:
ticker ="PETR3.SA"

acao = yf.download(ticker)
acao_regressao = acao["2022-08":"2022-11"]

acao_regressao["Date"] = acao_regressao.index

[*********************100%***********************]  1 of 1 completed


C:\Users\Facilimpa\AppData\Local\Temp\ipykernel_5520\1768267942.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  acao_regressao["Date"] = acao_regressao.index


In [42]:
SAR = psar(acao_regressao,0.2,0.2)
SAR_df = pd.DataFrame(SAR)


fig = go.Figure()

fig.add_trace(
    go.Scatter(
        name="Fechamento",
        x=SAR_df["dates"],
        y=SAR_df["close"],
        line=dict(color='blue', width=2))
)

fig.add_trace(
    go.Scatter(
        name="SAR bear",
        x=SAR_df["dates"],
        y=SAR_df["psarbear"],
        line=dict(color='red', width=1))
)

fig.add_trace(
    go.Scatter(
        name="SAR bull",
        x=SAR_df["dates"],
        y=SAR_df["psarbull"],
        line=dict(color='blue', width=1))
)



fig.show()